In [1]:
from arcgis.gis import GIS, Item
import os

item_id = 'b23955c522ec47548dc355bb5fd79d1c'
field_store_id_alias = 'Store ID'
file_root = 'demo_trips'

In [2]:
gis = GIS(url='https://commteamretail.maps.arcgis.com')
gis

In [3]:
feature_layer = Item(gis, item_id).layers[0]
feature_layer

<FeatureLayer url:"https://services.arcgis.com/PMTtzuTB6WiPuNSv/arcgis/rest/services/National_Store_Locations/FeatureServer/0">

In [4]:
field_store_id = [field.name for field in feature_layer.properties.fields if field.alias == field_store_id_alias][0]
field_store_id

'NationalStoreLocations_Local__3'

In [5]:
feature_set = feature_layer.query(out_fields=field_store_id, out_sr={'wkid': 4326})
feature_set

<FeatureSet> 3156 features

In [6]:
sdf = feature_set.df
sdf.drop('OBJECTID', axis=1, inplace=True)
sdf.columns = ['store_id', 'SHAPE']
sdf.head()

,store_id,SHAPE
0,96,"{'x': -86.49607497200131, 'y': 39.155960926931..."
1,97,"{'x': -122.18647465212695, 'y': 47.71139930792..."
2,98,"{'x': -122.10074581765247, 'y': 37.65401004768..."
3,99,"{'x': -76.83241134100487, 'y': 39.277317841688..."
4,100,"{'x': -119.32970923096121, 'y': 36.22602745380..."


In [7]:
from inrix import TAZ
import json

credentials_path = './resources/credentials.json'
credentials = json.load(open(credentials_path))

In [8]:
taz = TAZ(
    app_id=credentials['inrix']['app_id'],
    hash_token=credentials['inrix']['hash_id']
)
taz

In [9]:
def get_exisiting(directory, file_root):
    f_list = [f for f in os.listdir(directory) if os.path.isfile(f) and f.endswith('.csv') and f.startswith(file_root)]
    f_list = [f.split('.')[0] for f in f_list]
    return [int(f.split('_').pop()) for f in f_list]

In [ ]:
locnum_list = [store_id for store_id in sdf.store_id if store_id not in get_exisiting('.', file_root)]
remaining_count = len(locnum_list)
print('Remaining location count: {}'.format(remaining_count))

for index, store_id in enumerate(locnum_list):
    
    row = sdf.loc[sdf.store_id == store_id].iloc[0]
    
    sdf_this = taz.get_trip_destination_spatial_dataframe(row.SHAPE.y, row.SHAPE.x, '300m')
    sdf_this['store_id'] = row.store_id
    
    sdf_this.to_csv('demo_trips_{}.csv'.format(row.store_id))
    
    print('Retrieved and saved for Store ID {} - {}/{}'.format(row.store_id, index+1, remaining_count))

Remaining location count: 2504
Retrieved and saved for Store ID 748 - 1/2504
Retrieved and saved for Store ID 749 - 2/2504
Retrieved and saved for Store ID 750 - 3/2504
Retrieved and saved for Store ID 751 - 4/2504
Retrieved and saved for Store ID 752 - 5/2504
Retrieved and saved for Store ID 753 - 6/2504
Retrieved and saved for Store ID 754 - 7/2504
Retrieved and saved for Store ID 755 - 8/2504
Retrieved and saved for Store ID 756 - 9/2504
Retrieved and saved for Store ID 757 - 10/2504
Retrieved and saved for Store ID 758 - 11/2504
Retrieved and saved for Store ID 759 - 12/2504
Retrieved and saved for Store ID 760 - 13/2504
Retrieved and saved for Store ID 761 - 14/2504
Retrieved and saved for Store ID 762 - 15/2504
Retrieved and saved for Store ID 763 - 16/2504
Retrieved and saved for Store ID 764 - 17/2504
Retrieved and saved for Store ID 765 - 18/2504
